In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 

from sklearn import metrics
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import r2_score

In [2]:
org_data = pd.read_excel(r'all_mapped_data.xlsx',sheetname='prep_data')

In [3]:
org_data.columns = ['month-year', 'timeStamp', 'Date', 'Omr',
       'Platts', 'Forex', 'intra_volume', 'Freigth', 'inland_sales_ado',
       'inland_sales_HO50', 'ref_pdn_ado', 'ref_prod_H050']

In [4]:
org_data = org_data[['timeStamp','Omr','Platts']]

In [5]:
org_data.shape

(740, 3)

In [6]:
data = org_data

In [7]:
data['Omr_lag1'] = data['Omr'].shift(1)
data['platts_lag1'] = data['Platts'].shift(1)
data['platts_lag2'] = data['Platts'].shift(2)
data['platts_diff'] = data['platts_lag1'] - data['platts_lag2']

In [8]:
data = data.loc[:,['timeStamp','Omr','Omr_lag1', 'platts_diff']]
data = data.loc[data.platts_diff != 0 ,]

In [9]:
data.shape

(732, 4)

In [10]:
stDate = pd.to_datetime("01-08-2018",format="%d-%m-%Y")
endDate = pd.to_datetime("31-12-2018",format="%d-%m-%Y")
data = data.query('timeStamp >= @endDate | timeStamp <= @stDate')
#data = data.query('timeStamp <= @stDate')
data.dropna(inplace=True)
data.reset_index(inplace=True,drop=True)

In [11]:
data.shape

(603, 4)

In [12]:
data.head()

timeStamp    Omr  Omr_lag1  platts_diff
0 2016-12-05  468.9     455.9         1.50
1 2016-12-06  460.7     468.9         1.25
2 2016-12-07  450.0     460.7        -7.00
3 2016-12-08  447.6     450.0        -2.00
4 2016-12-09  459.5     447.6        -1.00

In [13]:
r2_model = []
pred_omr = []
omr = []
abs_error = []
date_col = [] 
for idx in range (len(data)-61):
    tmp = data.iloc[idx:idx+60,]
    #print("shape of the array {}& idx {} ".format(tmp.shape,idx+60))
    regr = LinearRegression()
    X_train = tmp.loc[:,['Omr_lag1','platts_diff']]
    y_train = tmp.loc[:,'Omr']
    regr.fit(X_train, y_train)
    r2 = r2_score(y_train, regr.predict(X_train), multioutput='variance_weighted')
    r2_model.append(r2)
    y_pred = regr.predict(data.loc[idx+60:,['Omr_lag1','platts_diff']])
    pred_omr.append(y_pred)
    omr.append(data.loc[idx+60:,'Omr'].values)
    abs_error.append(abs(y_pred - data.loc[idx+60:,'Omr'].values))
    date_col.append(data.loc[idx+60:,'timeStamp'].values)

In [94]:
# res_df = pd.DataFrame({'timeStamp':date_col[0],'omr':omr[0],
#                        'pred_omr':pred_omr[0],'abs_error':abs_error[0],'r2_model':r2_model})

In [102]:
res_df = pd.DataFrame({'timeStamp':date_col[0],'omr':omr[0],
                       'pred_omr':pred_omr[0],'abs_error':abs_error[0]})

In [103]:
res_df.head(20)

abs_error    omr    pred_omr  timeStamp
0    4.943207  471.3  476.243207 2017-03-08
1   16.218156  454.7  470.918156 2017-03-09
2    3.771113  450.0  453.771113 2017-03-10
3   17.562998  438.2  455.762998 2017-03-13
4    9.454734  437.0  446.454734 2017-03-14
5    2.700256  442.3  445.000256 2017-03-15
6   12.503728  440.5  453.003728 2017-03-16
7   12.466547  435.8  448.266547 2017-03-20
8    7.924127  438.2  446.124127 2017-03-21
9   14.115364  432.2  446.315364 2017-03-22
10   9.212269  432.2  441.412269 2017-03-23
11  11.790618  432.2  443.990618 2017-03-24
12  15.445341  427.5  442.945341 2017-03-27
13   5.877380  434.6  440.477380 2017-03-28
14   4.208628  442.9  447.108628 2017-03-29
15   3.537265  448.8  452.337265 2017-03-30
16   3.704980  453.6  457.304980 2017-03-31
17   3.181155  455.9  459.081155 2017-04-03
18   5.248824  455.9  461.148824 2017-04-04
19   3.778733  466.6  462.821267 2017-04-05

In [104]:
res_df.abs_error.mean()

14.16805016393656

In [98]:
regr.coef_

array([ 0.75108991,  0.34176032])

In [ ]:
regr = LinearRegression() 
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

print(metrics.mean_absolute_error(y_test,y_pred))
print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

In [14]:
r2_model = []
pred_omr = []
omr = []
abs_error = []
date_col = [] 
params = []
for idx in range (len(data)-61):
    tmp = data.iloc[idx:idx+60,]
    #print("shape of the array {}& idx {} ".format(tmp.shape,idx+60))
    regr = LinearRegression()
    X_train = tmp.loc[:,['Omr_lag1','platts_diff']]
    y_train = tmp.loc[:,'Omr']
    
    #X_train = sm.add_constant(X_train)
    model = sm.OLS(y_train, X_train).fit()
    params.append(dict(model.params))
    r2 = r2_score(y_train, model.predict(X_train), multioutput='variance_weighted')
    r2_model.append(r2)
    X_test = data.loc[idx+60:,['Omr_lag1','platts_diff']]
    y_pred = model.predict(X_test)
    pred_omr.append(y_pred)
    omr.append(data.loc[idx+60:,'Omr'].values)
    abs_error.append(abs(y_pred - data.loc[idx+60:,'Omr'].values))
    date_col.append(data.loc[idx+60:,'timeStamp'].values)

In [15]:
res_df = pd.DataFrame({'timeStamp':date_col[0],'omr':omr[0],
                       'pred_omr':pred_omr[0],'abs_error':abs_error[0]})

In [16]:
r2_model

[0.5618653787653608,
 0.58375561875232573,
 0.57169269740130202,
 0.58430949635515017,
 0.60475663282333647,
 0.68744033270146465,
 0.77433507541507329,
 0.81199488825591937,
 0.83643978612726522,
 0.85163594010657029,
 0.86863862048513674,
 0.88191601825371624,
 0.89231732686731502,
 0.90141936406047418,
 0.91006368614137767,
 0.91109576166316741,
 0.91565810080463528,
 0.9135941700930813,
 0.91129432302419999,
 0.91063342933697244,
 0.91746905998883754,
 0.91742048969079293,
 0.91872683130391652,
 0.91974312471045583,
 0.92063171470674665,
 0.92085944966972633,
 0.9243207288122729,
 0.92279682192026924,
 0.92290884051616429,
 0.91870621198128632,
 0.92081677666146444,
 0.91977132657522598,
 0.92270476457964068,
 0.92656494814082602,
 0.92896314915063316,
 0.92993782468626596,
 0.93073911295068334,
 0.93998510839183702,
 0.94276031461291965,
 0.94736773024671928,
 0.94781230796396398,
 0.94749724083242626,
 0.95038624937238314,
 0.94498527932157739,
 0.94459770145995892,
 0.9427064687

In [160]:
data.describe()

index         Omr    Omr_lag1  platts_diff
count  603.000000  603.000000  603.000000   603.000000
mean   354.210614  500.979270  500.848590     0.388474
std    223.712181   53.781354   53.877877     8.724554
min      2.000000  381.400000  381.400000   -29.250000
25%    163.500000  460.700000  460.700000    -4.750000
50%    317.000000  504.400000  500.900000     1.000000
75%    577.500000  545.900000  545.900000     5.875000
max    739.000000  607.400000  607.400000    59.750000

In [161]:
pd.__version__

'0.19.2'

In [162]:
! pip install --upgrade pandas

  Found existing installation: pytz 2016.10
    Uninstalling pytz-2016.10:


    DEPRECATION: Uninstalling a distutils installed project (pytz) has been deprecated and will be removed in a future version. This is due to the fact that uninstalling a distutils project will only partially uninstall the project.
Exception:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\shutil.py", line 538, in move
    os.rename(src, real_dst)
PermissionError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\pytz-2016.10-py3.6.egg-info' -> 'C:\\Users\\schatt8\\AppData\\Local\\Temp\\pip-prn9ati4-uninstall\\programdata\\anaconda3\\lib\\site-packages\\pytz-2016.10-py3.6.egg-info'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\basecommand.py", line 215, in main
    status = self.run(options, args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\commands\install.py", line 342, in run
    prefix=options.prefix_path